# SOME BACKGROUND

~ STEP 1 :- First download the latest Python Veriosn 
            https://www.python.org/downloads/

Please Install the downloaded file on your computer using custom Installation settings. 

~ STEP 2 :- After the installation is done , Open  , In MAC - Terminal or WIN - Terminal , and type "Python3 --version"
 for MAC or "Python --version" for WIN and press Enter 

 If , the Terminal , returns with the verion of Python like "Python 3.11.1" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 1 has been accomplished

~STEP 3 :- After we are done  with the STEP 2 , I am using the Jupytr Lab for the tutorials. , type in terminal "pip install jupyter lab" , this will install the latest verison of Jupyter Lab on your system , automatically you will be redirected to the browser 

~ STEP 4 :- To check , if the installation of Jupyter lab has been successful , type "jupyter lab --version" on Terminal. If , the Terminal , returns with the verion of Jupyter Lab like "4.1.2" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 3 has been accomplished

~ STEP 5 :- A library is a collection of code that makes everyday tasks more efficient. For my tutorial i would recommend Installing pandas , numpy - For data analysis , seaborn , matplotlib - For Visualisation.  

Also , i would be Installing a library called "peakutils" , this woulbe be useful for recogisation of Peaks from the Spectra. https://peakutils.readthedocs.io/en/latest/


~ STEP 6 :- For installing the library , go to terminal , and type "pip install pandas" and after installing . Similarly , do this for numpy , seaborn , matplotlin and PeakUtils

~ STEP 7 :- Now , to open the Jupyter Lab seession in the browser , go to terminal and type "jupyter lab" , you will be automatically redirected to the browser in a jupyter lab session. This is where you will start programming . 

NOTE :- To exit the session , press "Ctrl + C" on the termianal , this will end the running session on your system

~ STEP 8 :- In jupyter lab , your lines of codes or chunk of lines of codes  are called as "Cell" , If the "Cell are of type Markdown - Used for Comments / Information " and If the "Cells are of type Code - Used for Programming" 

You can get more in fo here -  https://www.datacamp.com/tutorial/installing-jupyter-notebook


# Importing the Useful librararies. 

Line [1] is completely optional , This is used to Hide the the Warnings in the Output cell. It's recommended to skip the line[1] , and proceed with the code from line[2].

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show

from bokeh.io import output_notebook
output_notebook()


Loading BokehJS ...

# LOADING THE SPECTRA FILE WHICH WE HAVE

In [3]:
file_path = 'New_Spectra/CR240LA/CR240LA_1.csv'

Sample_Spectra = pd.read_csv(file_path)
Sample_Spectra = Sample_Spectra.rename({' intensity':'Intensities'},axis=1)


Sample_Spectra_Plot = figure(title = 'Raw Sample Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Plot.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Plot.width = 1200
Sample_Spectra_Plot.height = 600


print (Sample_Spectra)
show (Sample_Spectra_Plot)



       wavelength  Intensities
0      180.000000          0.0
1      180.033333          0.0
2      180.066667          0.0
3      180.100000          0.0
4      180.133333          0.0
...           ...          ...
23426  960.866667          0.0
23427  960.900000          0.0
23428  960.933333          0.0
23429  960.966667          0.0
23430  961.000000          0.0

[23431 rows x 2 columns]


##### Now , as we see in the figure avove , the spectra have lots of lines , which are very specific to the element present in the Sample . 

Each element has an specific , wavelength associated with it.

Now , i have visited the NIST LIBS database , and download the spectra for all the element in the Periodic Table. The File are updated with that of NIST database as of 17 April 2024.

# CHOOSING THE ELEMENT OF INTEREST

In [4]:

element = 'Fe' # ENTER YOUR ELEMENT OF CHOICE
Reference_Spectra = pd.read_csv(r"NIST/" + element +".csv" , encoding = 'windows - 1252' , usecols=(0,1))
print(Reference_Spectra)

Reference_Spectra_Plot = figure(title = 'NIST Reference Library ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Plot.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Plot.width = 1200
Reference_Spectra_Plot.height = 600

show(Reference_Spectra_Plot)

       Wavelength       Spectra
0          169.08  2.630000e-10
1          169.12  5.470000e-09
2          169.16  1.020000e-07
3          169.20  1.700000e-06
4          169.24  2.550000e-05
...           ...           ...
16653      963.36  9.000000e+01
16654      963.40  1.090000e+02
16655      963.44  1.310000e+02
16656      963.48  1.570000e+02
16657      963.52  1.870000e+02

[16658 rows x 2 columns]


# APPROACH FOR PEAK DETECTION IN THE NIST ELEMENT SPECTRA

Now, uptil now , we have selected the Material Sample ,which we want to Interact with and the data is stored in variable "Sample_Spectra"
Also , we have choosen the element , which we want to detect in this 'Sample_Spectra' , and the spectral information for that element is stored in variable "Reference_Spectra" 

Now , from this 'Reference_Spectra' , Let's try to get all the Peaks this Spectra has

I have used a library called as Peak_Utils 

https://peakutils.readthedocs.io/en/latest/reference.html

The detail of the function is given below. 

---------------------------------------------------------------------------------------------------------------------------

NOTE :- Now if we keep the minimum_distance_1 = 0 and threshold_1 = 0 , we get all the peaks in the spectra , Tuning of these parameter are most important as per your signal.

Setting the value in parameter 'minimum_distance_1' , will allows to skip the peaks , which are very close to each other. 
Setting the value in parameter 'threshold_1', will allows to skip the peaks with low intensity and also seperate the Peaks , with could be the noise signal. 

So , its important to choose the value's as per your convenience 

----------------------------------------------------------------------------------------------------------------------------

In [16]:
minimum_distance_1 = 0 
threshold_1 = 0.01 #looking for a value above 0.01% , so that Peak with very low Intensities can be ignored 

Reference_Peak = indexes(Reference_Spectra.Spectra , min_dist=minimum_distance_1 , thres=threshold_1) 

Selected_Reference_Spectra = Reference_Spectra.iloc[Reference_Peak] #from the indices , lets try to find the Wavelength and Spectra. 

Reference_Spectra_Peaks = figure(title = 'Peaks in NIST Spectra ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Peaks.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Peaks.circle(Selected_Reference_Spectra.Wavelength , Selected_Reference_Spectra.Spectra , color = 'red' , size = 8)

Reference_Spectra_Peaks.width = 1200
Reference_Spectra_Peaks.height = 600

print(Selected_Reference_Spectra)
show(Reference_Spectra_Peaks)



      Wavelength       Spectra
27        170.19  1.630000e+09
44        170.89  3.690000e+08
54        171.30  9.280000e+08
72        172.04  6.800000e+08
85        172.57  3.590000e+08
...          ...           ...
3739      322.71  3.010000e+08
4601      358.13  4.720000e+08
4976      373.53  4.550000e+08
5008      374.85  3.080000e+08
5182      382.00  2.980000e+08

[70 rows x 2 columns]


# Wavelengths for Detected Peak(red circles above)

In [17]:
Reference_Spectra_List = []

for iteam in Selected_Reference_Spectra.Wavelength:
    iteam = float(iteam)
    if iteam > 187.0:        #Only those iteam will be selected , whose wavelength would be greater than 188 nm ,  as our Handheld LIBS range starts from 188nm
        Reference_Spectra_List.append(iteam)
    
print(Reference_Spectra_List) #These are the wavelengths of Peaks , which we are interested in the NIST Spectra
Length_Reference_Spectra_List = (len(Reference_Spectra_List))   #Used for Calculating Percentage Detection

[187.73, 188.88, 200.02, 216.7, 222.04, 224.88, 225.12, 233.26, 233.79, 234.37, 234.78, 236.01, 236.46, 237.41, 238.19, 238.85, 239.59, 240.53, 241.07, 242.42, 243.28, 243.94, 244.52, 246.2, 247.19, 248.3, 249.24, 249.9, 251.17, 252.65, 253.8, 254.91, 256.27, 257.71, 258.57, 259.23, 259.92, 260.75, 261.2, 263.09, 266.54, 268.47, 269.25, 270.44, 271.43, 272.74, 273.98, 274.8, 275.54, 276.77, 278.37, 284.0, 294.73, 298.47, 300.19, 302.08, 322.71, 358.13, 373.53, 374.85, 382.0]


# 0.0333 is the sprectral resolution of the Handheld LIBS Device

Now , for the two wavelengths from two different data source , to exactly match with each other , is going to be difficult because of the spectral resolution of wavelength - for these two data source are very different from each other.

Now we know that the Spectral Resolution of Handheld LIBS Device is 0.0333. 

Now , we have a list of all wavelengths , detected for Peaks in Reference Spectra , Now for each element lets try to create a min and max of each element in the list. 'Reference_Spectra_Min_Max' is list consisting of a sublist , the sublist have a min and max value or a range of element in the list.


Eg : - A peak of 254.8 in NIST Spectra , can be found at and between [254.7668 ~ 254.8332] in the Handheld LIBS Spectra , because the resolution of wavelength in NIST Spectra and Handheld LIBS Spectra is different. 


In [18]:
Reference_Spectra_Min_Max = []
Reference_Spectra_Min_Max = [[num - 0.0332, num + 0.0332] for num in Reference_Spectra_List]     
    
print(Reference_Spectra_Min_Max)

[[187.6968, 187.76319999999998], [188.8468, 188.9132], [199.98680000000002, 200.0532], [216.6668, 216.73319999999998], [222.0068, 222.07319999999999], [224.8468, 224.9132], [225.0868, 225.1532], [233.2268, 233.29319999999998], [233.7568, 233.82319999999999], [234.3368, 234.4032], [234.7468, 234.8132], [235.9768, 236.04319999999998], [236.42680000000001, 236.4932], [237.3768, 237.4432], [238.1568, 238.2232], [238.8168, 238.8832], [239.5568, 239.6232], [240.4968, 240.5632], [241.0368, 241.1032], [242.3868, 242.45319999999998], [243.2468, 243.3132], [243.9068, 243.9732], [244.48680000000002, 244.5532], [246.1668, 246.23319999999998], [247.1568, 247.2232], [248.26680000000002, 248.3332], [249.20680000000002, 249.2732], [249.8668, 249.9332], [251.1368, 251.20319999999998], [252.6168, 252.6832], [253.76680000000002, 253.8332], [254.8768, 254.9432], [256.23679999999996, 256.3032], [257.67679999999996, 257.7432], [258.53679999999997, 258.6032], [259.1968, 259.26320000000004], [259.8868, 259.95

# Pre-Processing the Spectra for Handheld LIBS to improve the accuracy of Peak Detection 

As we see in our raw spectra , there are lot of small peaks , which really aren't emission peaks , but are the noise in the signal. To tackle that Smoothing the Spectra is usually done. 

I am sing the Savitsky Golay , for smoothing my spectra. 

# Noise Estimation
As we see in the Handheld LIBS Spectra , It's difficult to find or detect , what is the noise and what is the signal.

But we have an information , that the range of Spectrometer in hLIBS starts from 190nm , So we can say that the signal detected uptil 190nm could be classified as noise.

In [19]:
noise_list = [] 

noise_list = Sample_Spectra.loc[Sample_Spectra['wavelength'] < 190.0 , 'Intensities']

# RMS_intensity_noise = max(noise_list) 
RMS_intensity_noise = np.sqrt(np.mean((noise_list)**2)) #This is RMS of the noise signal

print(RMS_intensity_noise)  



86.32554474497412


In [20]:
#METHOD  - Scipy

Threshold = 4 #(Noise Intensity * Threshold) Value for signal filtering
height_2 = None
Peak_Prominenace = (Threshold * RMS_intensity_noise) # #Uses S/N ratio , as comapred to other peaks
minimum_distance_3 = None   #minimum distance between two peaks

Sample_Peak , _ = fp(Sample_Spectra.Intensities,height=height_2,prominence=Peak_Prominenace,distance=minimum_distance_3)
Selected_Sample_Spectra = Sample_Spectra.iloc[Sample_Peak] 

Sample_Spectra_Peaks = figure(title = 'Peaks in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Peaks.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Peaks.circle(Selected_Sample_Spectra.wavelength , Selected_Sample_Spectra.Intensities , color = 'red' , size = 8)

Sample_Spectra_Peaks.width = 1200
Sample_Spectra_Peaks.height = 600

print(Selected_Sample_Spectra)
show(Sample_Spectra_Peaks)

       wavelength   Intensities
266    188.866667    451.545855
392    193.066667    709.445099
570    199.000000   1449.789347
601    200.033333   1227.880788
623    200.766667    750.249665
...           ...           ...
19874  842.466667  12086.914303
20164  852.133333   4333.762936
20603  866.766667    948.455353
21968  912.266667   5315.454182
22272  922.400000   1970.989893

[424 rows x 2 columns]


# Wavelengths for Detected Peak(red circles above)

In [21]:
Sample_Spectra_List = []

for iteam in Selected_Sample_Spectra.wavelength:
    iteam = float(iteam)
    Sample_Spectra_List.append(iteam)
    
print(Sample_Spectra_List)


[188.86666666666616, 193.06666666666592, 198.9999999999989, 200.0333333333322, 200.7666666666655, 201.06666666666547, 202.56666666666536, 204.0666666666653, 206.1999999999985, 206.3999999999985, 206.9999999999985, 207.93333333333172, 208.6999999999984, 209.39999999999836, 209.6999999999983, 209.9999999999983, 210.19999999999828, 210.4333333333316, 211.0666666666649, 213.86666666666477, 215.06666666666467, 216.1666666666646, 216.66666666666455, 217.5333333333312, 217.7666666666645, 218.6999999999978, 219.16666666666444, 219.5666666666644, 220.03333333333103, 220.86666666666437, 221.36666666666432, 222.03333333333092, 222.36666666666423, 222.8333333333309, 223.39999999999756, 224.89999999999745, 225.19999999999743, 225.56666666666408, 225.99999999999736, 226.3333333333307, 226.73333333333068, 227.99999999999727, 228.36666666666392, 229.3999999999972, 229.76666666666384, 231.13333333333043, 232.73333333333036, 233.2333333333303, 233.7666666666636, 234.3666666666636, 234.79999999999688, 23

# So far so now , i have two list with me 

1) Sample_Spectra_List = Containg wavelngths of all Peaks from data of Handheld LIBS device
2) Reference_Spectra_Min_Max = Containing a range of wavelength , for all the peak detected in the Reference Sample

   

In [22]:
print(Sample_Spectra_List)

[188.86666666666616, 193.06666666666592, 198.9999999999989, 200.0333333333322, 200.7666666666655, 201.06666666666547, 202.56666666666536, 204.0666666666653, 206.1999999999985, 206.3999999999985, 206.9999999999985, 207.93333333333172, 208.6999999999984, 209.39999999999836, 209.6999999999983, 209.9999999999983, 210.19999999999828, 210.4333333333316, 211.0666666666649, 213.86666666666477, 215.06666666666467, 216.1666666666646, 216.66666666666455, 217.5333333333312, 217.7666666666645, 218.6999999999978, 219.16666666666444, 219.5666666666644, 220.03333333333103, 220.86666666666437, 221.36666666666432, 222.03333333333092, 222.36666666666423, 222.8333333333309, 223.39999999999756, 224.89999999999745, 225.19999999999743, 225.56666666666408, 225.99999999999736, 226.3333333333307, 226.73333333333068, 227.99999999999727, 228.36666666666392, 229.3999999999972, 229.76666666666384, 231.13333333333043, 232.73333333333036, 233.2333333333303, 233.7666666666636, 234.3666666666636, 234.79999999999688, 23

In [23]:
print(Reference_Spectra_Min_Max)

[[187.6968, 187.76319999999998], [188.8468, 188.9132], [199.98680000000002, 200.0532], [216.6668, 216.73319999999998], [222.0068, 222.07319999999999], [224.8468, 224.9132], [225.0868, 225.1532], [233.2268, 233.29319999999998], [233.7568, 233.82319999999999], [234.3368, 234.4032], [234.7468, 234.8132], [235.9768, 236.04319999999998], [236.42680000000001, 236.4932], [237.3768, 237.4432], [238.1568, 238.2232], [238.8168, 238.8832], [239.5568, 239.6232], [240.4968, 240.5632], [241.0368, 241.1032], [242.3868, 242.45319999999998], [243.2468, 243.3132], [243.9068, 243.9732], [244.48680000000002, 244.5532], [246.1668, 246.23319999999998], [247.1568, 247.2232], [248.26680000000002, 248.3332], [249.20680000000002, 249.2732], [249.8668, 249.9332], [251.1368, 251.20319999999998], [252.6168, 252.6832], [253.76680000000002, 253.8332], [254.8768, 254.9432], [256.23679999999996, 256.3032], [257.67679999999996, 257.7432], [258.53679999999997, 258.6032], [259.1968, 259.26320000000004], [259.8868, 259.95

# Finally , My  idea is see , if each iteam in (Reference_Spectra_Min_Max) , matches with each element in (Sample_Spectra_List) , and then caluclate the exact matching Percentage 


In [24]:
plot_spectra_wavelength_list=[]
plot_reference_wavelength_list = []
count = 0
for cell in Reference_Spectra_Min_Max: 
    for iteam in Sample_Spectra_List:
        if iteam >= cell[0] and iteam <= cell[1]:
            count = count + 1 #for counting
            plot_spectra_wavelength_list.append(iteam)
            plot_reference_wavelength_list.append(cell[0] + 0.0333)

plot_spectra_intensities_list=[]
for iteam in plot_spectra_wavelength_list:
    Inte = Selected_Sample_Spectra.loc[Selected_Sample_Spectra['wavelength'] == iteam, 'Intensities'].iloc[0]
    plot_spectra_intensities_list.append(Inte)

#print(plot_spectra_intensities_list)
#print(plot_spectra_wavelength_list)

print('The total peaks detected were ' + str(count) + '\n' )

print('The following peaks , for Reference Sample was detected  :- '  )
print(plot_reference_wavelength_list )

print('\n')
print('The Sample Peaks detected from the sample were:-')
spectra_df = pd.DataFrame({'wavelength': plot_spectra_wavelength_list, 'intensities': plot_spectra_intensities_list})
print(spectra_df)  



# MAX_Intensity = max(spectra_df.intensities)

# # ###########################################################################

# matching_row = Sample_Spectra[Sample_Spectra['Intensities'] == MAX_Intensity]
# print(matching_row)

The total peaks detected were 23

The following peaks , for Reference Sample was detected  :- 
[188.8801, 200.0201, 222.0401, 224.8801, 233.2601, 233.7901, 234.3701, 234.7801, 236.4601, 238.1901, 242.4201, 254.9101, 256.27009999999996, 257.71009999999995, 259.2301, 263.09009999999995, 271.4301, 275.5401, 284.0001, 294.7301, 298.4701, 358.13009999999997, 374.8501]


The Sample Peaks detected from the sample were:-
    wavelength   intensities
0   188.866667    451.545855
1   200.033333   1227.880788
2   222.033333   2356.668569
3   224.900000   2373.900887
4   233.233333  10020.804556
5   233.766667   9669.814687
6   234.366667  19787.271201
7   234.800000  19823.469359
8   236.466667  11652.448999
9   238.166667  23100.676747
10  242.400000   8717.207664
11  254.900000  13523.228796
12  256.266667  16840.141698
13  257.733333   6193.016405
14  259.233333   8153.279470
15  263.066667  24556.268954
16  271.400000   7720.939435
17  275.533333  20164.043915
18  284.000000   3465.472751
19 

# RESULTS

In [14]:
percentage = (count / Length_Reference_Spectra_List) * 100
percentage

print (' Ananlysis of ' + element + ' was done , and the ' + str(count) +  ' number of spectral lines from the material sample ' + ' matched with the ' + str(Length_Reference_Spectra_List) + ' choosen Reference element of '+ element + ' therefore , we can say' ,  str(percentage) + '%' + ' matched as per the parameters set by the user')

 Ananlysis of Fe was done , and the 13 number of spectral lines from the material sample  matched with the 35 choosen Reference element of Fe therefore , we can say 37.142857142857146% matched as per the parameters set by the user


Find the Correponding Intensities , for the wavelengths available in "plot_spectra_wavelength_list" and also for the wavelengths available in "plot_refernce_intensities_list"

# Generate a Interactive Plot

In [15]:
p = figure(title = 'Peak Detection' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensities')
p.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
p.circle(spectra_df.wavelength, spectra_df.intensities , color = 'green' , size = 5)

p.width = 1200
p.height = 600

show(p)

